In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from sodapy import Socrata

In [2]:
load_dotenv()

pd.set_option("display.max_columns", None)

## Grab Data from endpoint

In [83]:
# api endpoint
url = "https://data.sfgov.org/api/v3/views/nuek-vuh3/query.csv"

client = Socrata(
    "data.sfgov.org",
    os.getenv("SFGOV_APP_TOKEN"), 
    username=os.getenv("SFGOV_EMAIL"), 
    password=os.getenv("SFGOV_PASSWORD")
)

# First 1000000 results
results = client.get("nuek-vuh3", limit=1000000)

# Convert to dataframe
results_df = pd.DataFrame.from_records(results)

KeyboardInterrupt: 

## Save results

In [18]:
results_df.to_parquet('sample_ems_data.parquet', index=False)

## Select needed Columns

In [28]:
df = results_df[['call_number', 'incident_number', 'received_dttm', 'case_location']]

## Separate date into components

In [31]:
# Convert to datetime dtype
df['datetime'] = pd.to_datetime(df['received_dttm'])

# Extract components
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['time'] = df['datetime'].dt.time 

# Drop original columns (not needed anymore)
df = df.drop(columns=['received_dttm', 'datetime'])

## Separate longitude/latitude into components

In [66]:
df['longitude'] = df['case_location'].apply(lambda x: x['coordinates'][0] if pd.notnull(x) else None)
df['latitude'] = df['case_location'].apply(lambda x: x['coordinates'][1] if pd.notnull(x) else None)

df = df.drop(columns=['case_location'])

## Round times (put the time into buckets)

In [101]:
df['hour'] = df['time'].apply(lambda time: time.hour)

# Drop time because we only need hour now
df.drop(columns=['time'])

# Reorder columns
df = df[['call_number', 'incident_number', 'year', 'month', 'day', 'hour', 'longitude', 'latitude']]

KeyError: 'time'

In [104]:
df

,call_number,incident_number,year,month,day,hour,longitude,latitude
0,160943727,16037460,2016,4,3,23,-122.419830,37.786358
1,161021964,16040565,2016,4,11,13,-122.422040,37.766540
2,160930738,16036742,2016,4,2,7,-122.416985,37.782420
3,160931745,16036856,2016,4,2,13,-122.417620,37.783780
4,160921757,16036446,2016,4,1,13,-122.390520,37.737540
...,...,...,...,...,...,...,...,...
999995,001770251,00053133,2000,6,25,18,-122.442300,37.788780
999996,001820249,00054658,2000,6,30,19,-122.394150,37.783504
999997,001720276,00051704,2000,6,20,21,-122.410060,37.788082
999998,001720070,00051505,2000,6,20,8,-122.411700,37.781178


In [103]:
df.to_parquet('../data/processed_ems_data.parquet', index=False)

# Possible Features to Add:
- is_weekend
- is_rush_hour
- population_density
- is_event_happening